In [152]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from math import log, e
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [153]:
#VERSIONE CON USO DEI MOTIF,DISCORD O ENTRAMBI E CON RIMOZIONE DEI CANDIDATI GIA SCELTI

In [157]:
def retrieve_all(Ts,window_size): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [155]:
#ogni motif e identificato da almeno due indici di partenza nella Ts, ne prendo uno solo rappresentativo
#genero poi struttura contenente gli indici di partenza di tutti i candidati

def candidateFilter(CandidatesList): 
    counterNumberMotif=0
    counterNumberDiscord=0
    l2=np.array([])
    for i in range (len(CandidatesList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(CandidatesList['Motif'].iloc[i])
        numDiscord=len(CandidatesList['Discord'].iloc[i])
        counterNumberDiscord+=numDiscord
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=CandidatesList['Motif'].iloc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista
            counterNumberMotif+=1
            
    
        CandidatesList['Motif'].iloc[i]=l2
        l2=np.array([]) #svuoto array
    
    return CandidatesList,counterNumberMotif,counterNumberDiscord

In [156]:
#
def buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord):
    CandidatesUsedList=pd.DataFrame(columns=['Used'],index=range(0,numberOfMotif+numberOfDiscord))
    boolList=[False] * (numberOfMotif+numberOfDiscord)
    CandidatesUsedList['Used']= boolList
    return CandidatesUsedList
                                    

In [196]:
def getDataStructures(df,window_size):
    #trasformo da stringa a numero il campo target 
    le = LabelEncoder()
    num_classes = le.fit_transform(df['target'])
    df['target']=num_classes
    df['TsIndex']=np.arange(len(df))
    window_size=5
    #diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
    diz={'Motif':[],'Discord':[]}

    #CALCOLO MOTIF E DISCORD E LI INSERISCO NEL DIZIONARIO
    for i in range(len(df)):
        Ts = np.array(df.iloc[i][0:-2].values)
        mp,mot,motif_dist,dis = retrieve_all(Ts,window_size)
        diz['Motif'].insert(i, mot)
        diz['Discord'].insert(i, dis)

    #GENERO DFRAME DA DIZIONARIO

    CandidatesList = pd.DataFrame(diz)
    CandidatesList,numberOfMotif,numberOfDiscord=candidateFilter(CandidatesList)
    CandidatesUsedList=buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord)

    print('Candidati estratti')
    #print(MotifsList['Motif'])
    print(CandidatesList)
    print(numberOfMotif,numberOfDiscord)
    print(CandidatesUsedList)
    
    return mp,CandidatesList,numberOfMotif,numberOfDiscord,CandidatesUsedList,num_classes

In [171]:
#per ogni Ts calcolo Dprofile con ogni candidato e inserisco la distanza minima con candidato i-esimo nella colonna i-esima
def computeSubSeqDistance(dataset,CandidatesList):
    
    #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    numberOfCandidates=0
    for i in range(len(CandidatesList)):
            numberOfCandidates+=len(CandidatesList['Motif'].loc[i])
            numberOfCandidates+=len(CandidatesList['Discord'].loc[i])
    columnsList=np.arange(numberOfCandidates)
    columnsList2=list()
    lastAttribute=['TsIndex','class']
    prefix='cand'
    for i in columnsList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(dataset)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(dataset)):
        #acquisisco la Ts
        TsToCompare = np.array(dataset.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for j in range(len(CandidatesList)):
            numMotif=len(CandidatesList['Motif'].iloc[j])
            numDiscord=len(CandidatesList['Discord'].iloc[j])
            for k in range(numMotif):
                l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
            for k in range(numDiscord):
                l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
        
   # print(counter)    
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                
                

In [172]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset, num_classes_unique):
    numPattern=len(dataset)
    if(numPattern>0):
        #print('numPattern: '+str(numPattern))
        relativeFrequencies=list()
        counter=0 #num pattern di ogni classe nel dataset
        #ottengo frequenze relative
        for i in (num_classes_unique):
            for j in range(numPattern):
                if(dataset.iloc[j]['class']==i):
                    counter+=1
            relativeFrequencies.append(counter/numPattern)
            counter=0
        #calcolo entropia dataset
        #print('relativeFrequencies: '+str(relativeFrequencies))
        entropy=0
        for k in (relativeFrequencies):
            if(k!=0):
                entropy+=(k*log(k,2))
        return -entropy
    else:
        return 0

In [173]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli
def computeGain(entropyParent,LenDatasetParent,Dleft,Dright,num_classes_unique):
    entropyLeft=computeEntropy(Dleft,num_classes_unique)
    entropyRight=computeEntropy(Dright,num_classes_unique)
    gain=entropyParent
    summation=( ((len(Dleft)/LenDatasetParent)*entropyLeft) +  ((len(Dright)/LenDatasetParent)*entropyRight) )
    #print('entropyParent: '+str(entropyParent))
    #print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [174]:
#SPLIT SLAVE
#effettua lo split del dataset sul attributo e valore fornito
def split(dataset,attribute,value): 
    columnsList=dataset.columns.values
    dizLeft=pd.DataFrame(columns=columnsList)
    dizRight=pd.DataFrame(columns=columnsList)
    for i in range(len(dataset)):
        if dataset.iloc[i][attribute] < value:
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
    return dizLeft, dizRight

In [206]:
# riceve vettore con mutual_information(gain) e in base al candidatesGroup scelto, determina il miglior attributo su cui splittare
# che non è stato ancora utilizzato
def getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedList):
    
    #definisco gli indici dei candidati da considerare, in base al candidatesGroup scelto
    
    if(candidatesGroup==0):
        candidatesIndex=range(numberOfMotif)
    elif(candidatesGroup==1):
        candidatesIndex=range(numberOfMotif,numberOfMotif+numberOfDiscord)
    else:
        candidatesIndex=range(numberOfMotif+numberOfDiscord)
    
    #genero struttura dati associando indice candidato e gain calcolato
    
    MutualInfoDf=pd.DataFrame(columns=['candidatesIndex','gain'],index=range(len(candidatesIndex)))
    MutualInfoDf['gain']=vecMutualInfo
    MutualInfoDf['candidatesIndex']=candidatesIndex
    
    #ordino i candidati in base a gain decrescente
    
    MutualInfoDf=MutualInfoDf.sort_values(by='gain',ascending = False)

    
    #scandisco i candidati fino a trovare il candidato con miglior gain che non è ancora stato usato
    
    bestIndexAttribute=-1
    i=0
    
    while(bestIndexAttribute==-1 and i<len(MutualInfoDf)):    
        attributeToVerify=int(MutualInfoDf.iloc[i]['candidatesIndex'])
        if(CandidatesUsedList.iloc[attributeToVerify]['Used']==False):
            bestIndexAttribute=attributeToVerify
            CandidatesUsedList.iloc[attributeToVerify]=True #settando a true il candidato scelto, non sarà usato in seguito
            print('BEST')
            print(bestIndexAttribute)
        else:
            i+=1
    
    return bestIndexAttribute
            
    
        
        

In [205]:
#SPLIT INTERMEDIO
#dato il dataset, cerca il miglior attributo e relativo valore (optimal split point) su cui splittare
# restituiendo il dataset splittato e i valori trovati
def findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedList):
    #cerca e restituisce attributo migliore su cui splittaree relativo valore ottimale (optimal split point)
    #CANDIDATE GROUP permette di scegliere se usare come candidati 0=motifs 1=discord 2=entrambi
    bestGain=0
    actualGain=0
    bestvalueForSplit=0
    y = dataset['class'].values
    y=y.astype('int')
    
    #trovo best Attribute
    numAttributes=len(dataset.columns.values)
    numAttributes-=2 #tolgo i due attributi TsIndex e class dal Dframe
    datasetForMutual=pd.DataFrame()
    
    #preparo il Dframe da passare a mutual_info_classif, settando se scegliere tra motifs/discord/entrambi
    
    if(candidatesGroup==0): #solo motifs
        datasetForMutual=dataset.iloc[:,np.r_[:numberOfMotif]]
    elif(candidatesGroup==1):
        datasetForMutual=dataset.iloc[:,np.r_[numberOfMotif:numberOfMotif+numberOfDiscord]]
    else:
        datasetForMutual=dataset.iloc[:,np.r_[:numAttributes]]
        
    vecMutualInfo=mutual_info_classif(datasetForMutual, y, discrete_features=False)
    indexBestAttribute=getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedList)
    
    
    num_classes_unique=np.unique(num_classes) 
    entropyParent=computeEntropy(dataset, num_classes_unique)
    DleftChosen=pd.DataFrame()
    DrightChosen=pd.DataFrame()
    #splitto su tale attributo e trovo il valore ottimale testando su tutte le distanze calcolate tre le Ts e tale candidato 
    for i in range (len(dataset)):
        testValue=dataset.iloc[i][indexBestAttribute]
        if(testValue>0): #distanze non-negative, split inutile su tale valore
            Dleft,Dright=split(dataset,indexBestAttribute,testValue)
            actualGain=computeGain(entropyParent,len(dataset),Dleft,Dright,num_classes_unique)
            if(actualGain > bestGain):
                bestGain=actualGain
                bestvalueForSplit=testValue
                DleftChosen=Dleft
                DrightChosen=Dright

    
    return indexBestAttribute,bestvalueForSplit,DleftChosen,DrightChosen

In [217]:
#SPLIT MASTER
# funzione ricorsiva che implementa la creazione dell'albero di classificazione
# memorizza in ogni nodo: attributo, valore attributo su cui splitto, entropia nodo, num pattern
# memorizza in ogni foglia: entropia nodo, num pattern, classe nodo

# VERSIONE CHE RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 

def buildTree(actualNode,dataset,maxDepth, minSize, depth,num_classes_unique,candidatesGroup):
    #caso base: num pattern < soglia minima || profondità massima raggiunta => genero foglia con media delle classi
    #DATASET HA SEMPRE ALMENO UN PATTERN
    boolValue=checkIfIsLeaf(dataset)
    if(len(dataset)<minSize or depth>=maxDepth or boolValue==True ):
        average = sum(dataset['class'].values) / len(dataset['class'].values)
        classValue = round(average)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset, num_classes_unique)
        
        nodeInfo=list()
        nodeInfo.append(classValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
    
        actualNode.data=nodeInfo
        actualNode.value=-1
        actualNode.left=None
        actualNode.right=None
        #print(dataset['class'])
        return 
    #caso in cui si può splittare
    else:
        
        indexChosenAttribute,attributeValue,Dleft,Dright=findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedList)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset, num_classes_unique)
        
        #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
        nodeInfo=list()
        nodeInfo.append(attributeValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
        actualNode.data=nodeInfo
        actualNode.value=(indexChosenAttribute)
        
        #se possibile richiamo ricorsivamente sul nodo dx e sx figlio
        if(len(Dleft)>0):
            actualNode.left=Node(indexChosenAttribute)
            buildTree(actualNode.left,Dleft,maxDepth, minSize, depth+1,num_classes_unique,candidatesGroup)
        
        if(len(Dright)>0):
            actualNode.right=Node(indexChosenAttribute)
            buildTree(actualNode.right,Dright,maxDepth, minSize, depth+1,num_classes_unique,candidatesGroup)
            
        

In [218]:
def checkIfIsLeaf(dataset):
    isLeaf=True
    classValue=dataset.iloc[0]['class']
    for i in range(len(dataset)):
        if(classValue!=dataset.iloc[i]['class']):
            isLeaf=False
            break
    return isLeaf

In [219]:
#effettua il primo passo dell'algo di generazione dell'albero, richiama ricorsivamente sui figli
# VERSIONE CHE NON RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 
def startAlgo(dfForDTree,num_classes,candidatesGroup,CandidatesUsedList):
    num_classes_unique=np.unique(num_classes)
    
    #inizio algo per nodo radice
    indexChosenAttribute,attributeValue,Dleft,Dright=findBestAttributeValue(dfForDTree,candidatesGroup,CandidatesUsedList)
    root=Node(indexChosenAttribute)
    numPattern=len(dfForDTree)
    entropy=computeEntropy(dfForDTree, num_classes_unique)
        
    #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
    nodeInfo=list()
    nodeInfo.append(attributeValue)
    nodeInfo.append(numPattern)
    nodeInfo.append(entropy)
    root.data=nodeInfo
    
    root.left=Node(indexChosenAttribute)
    root.right=Node(indexChosenAttribute)
    
    #chiamata ricorsiva
    if(len(Dleft)>0):
        buildTree(root.left,Dleft,3, 3,1,num_classes_unique,candidatesGroup)
    if(len(Dright)>0):
        buildTree(root.right,Dright,3, 3, 1,num_classes_unique,candidatesGroup)
    return root

In [220]:
#stampa dell'albero
def printAll(Root):
    if(Root.left==None and Root.right==None):
        print('foglia')
    print('Nodo: '+str(Root.value))
    df=Root.data
    print(df)
    print("\n")
    if(Root.left!=None):
        printAll(Root.left)
    if(Root.right!=None):
        printAll(Root.right)

In [221]:
def predict(testDataset,root):
    print('START PRED')
    #preparo dataset 
    numAttributes=len(dfForDTree.columns.values)
    numAttributes-=2 #per prendere solo gli attributi utili a xTest
    yTest=testDataset.iloc[:]['class']
    yPredicted=pd.DataFrame(columns=['class'],index=range(len(yTest)))
    xTest=testDataset.iloc[:,np.r_[:numAttributes]]
    
    for i in range(len(xTest)):
        pattern=xTest.iloc[i]
        yPredicted.iloc[i]['class']= treeExplorer(pattern,root)
        
    counter=0
    for j in range (len(yTest)):
        if(yTest.iloc[j] == yPredicted.iloc[j]['class']):            
            counter+=1

    precision=counter/len(yTest)
    
    return precision,yTest,yPredicted['class']
        

In [222]:
def treeExplorer(pattern,node):
    #caso base, root è foglia
    if(node.value==-1):
        return int(node.data[0])
    else:
        attr='cand'+str(node.value)
        if(pattern[attr] < node.data[0]):
            return treeExplorer(pattern,node.left)
        else:
            return treeExplorer(pattern,node.right)

In [223]:
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
df=pd.DataFrame(dataset[0])
window_size=5
mp,CandidatesList,numberOfMotif,numberOfDiscord,CandidatesUsedList,num_classes=getDataStructures(df,window_size)
dfForDTree=computeSubSeqDistance(df,CandidatesList)


Candidati estratti
            Motif    Discord
0     [9.0, 26.0]   [61, 27]
1   [105.0, 54.0]   [8, 119]
2    [17.0, 40.0]    [31, 1]
3    [48.0, 12.0]   [23, 86]
4          [27.0]   [38, 62]
5    [45.0, 13.0]  [119, 26]
6     [7.0, 37.0]   [98, 76]
7     [2.0, 24.0]    [36, 8]
8     [1.0, 29.0]   [23, 32]
9     [50.0, 0.0]   [38, 59]
10   [11.0, 17.0]   [80, 23]
11    [31.0, 7.0]  [114, 54]
12         [53.0]   [61, 10]
13    [79.0, 6.0]  [105, 38]
14         [59.0]    [98, 8]
15    [2.0, 45.0]     [1, 8]
16    [28.0, 3.0]    [5, 88]
17   [14.0, 35.0]  [118, 85]
18    [13.0, 3.0]   [111, 8]
19    [19.0, 6.0]   [92, 35]
20   [19.0, 39.0]   [26, 70]
21         [26.0]   [13, 35]
22   [18.0, 22.0]   [59, 25]
23   [27.0, 42.0]  [32, 104]
24    [8.0, 28.0]    [4, 95]
25         [16.0]   [87, 20]
26   [67.0, 59.0]   [33, 83]
27   [16.0, 39.0]   [75, 99]
28   [11.0, 41.0]   [47, 39]
29    [6.0, 15.0]  [117, 51]
55 60
      Used
0    False
1    False
2    False
3    False
4    False
..     ...

In [224]:
candidatesGroup=1
albero=None
albero=startAlgo(dfForDTree,num_classes,candidatesGroup,CandidatesUsedList)
print(albero)
printAll(albero)


BEST
110
BEST
74
BEST
69
BEST
55
BEST
76
BEST
64

           __________110___
          /                \
     ____74___             _76___
    /         \           /      \
  _69         _55        -1      _64
 /   \       /   \              /   \
-1    -1    -1    -1           -1    -1

Nodo: 110
[0.5938733649571964, 30, 1.5655962303576019]


Nodo: 74
[0.6209200966233901, 14, 1.2638091738835462]


Nodo: 69
[0.5565313638815896, 7, 0.863120568566631]


foglia
Nodo: -1
[0.0, 2, -0.0]


foglia
Nodo: -1
[2.0, 5, -0.0]


Nodo: 55
[0.7887258019465716, 7, 0.5916727785823275]


foglia
Nodo: -1
[0.0, 4, -0.0]


foglia
Nodo: -1
[0.0, 3, 0.9182958340544896]


Nodo: 76
[0.36496414311772096, 16, 1.1994602933016414]


foglia
Nodo: -1
[2.0, 3, -0.0]


Nodo: 64
[0.5832963185506923, 13, 0.6193821946787638]


foglia
Nodo: -1
[1.0, 7, 0.863120568566631]


foglia
Nodo: -1
[1.0, 6, -0.0]




In [225]:
dataset2 = arff.loadarff('CBF/CBF_TEST.arff')
window_size=5
dataset2[0]
df2 = pd.DataFrame(dataset2[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
df2=df2.iloc[:50]
mp2,CandidatesList2,numberOfMotif2,numberOfDiscord2,CandidatesUsedList2,num_classes2=getDataStructures(df2,window_size)
candidatesGroup=0
dfForDTree2=computeSubSeqDistance(df2,CandidatesList2)
print(dfForDTree2)


precision,yTest,yPredicted=predict(dfForDTree2,albero)

print(precision)
for a,b in zip(yTest,yPredicted):
    print(int(a),b)

Candidati estratti
            Motif     Discord
0     [3.0, 25.0]    [93, 75]
1    [12.0, 19.0]    [117, 4]
2     [1.0, 61.0]    [93, 87]
3     [5.0, 33.0]   [75, 107]
4    [25.0, 32.0]    [29, 22]
5    [19.0, 52.0]   [87, 117]
6    [42.0, 46.0]   [24, 116]
7    [44.0, 14.0]    [21, 68]
8          [10.0]    [5, 119]
9     [5.0, 28.0]    [20, 93]
10    [16.0, 8.0]    [53, 94]
11   [24.0, 39.0]  [112, 106]
12   [77.0, 11.0]   [73, 101]
13         [51.0]    [68, 73]
14    [16.0, 9.0]   [120, 86]
15   [10.0, 50.0]   [77, 103]
16    [4.0, 74.0]     [84, 0]
17    [2.0, 48.0]   [95, 114]
18         [16.0]   [20, 115]
19   [28.0, 37.0]     [42, 7]
20    [28.0, 7.0]    [32, 97]
21   [31.0, 14.0]    [90, 65]
22   [49.0, 19.0]    [75, 64]
23    [8.0, 35.0]   [115, 90]
24    [40.0, 7.0]    [18, 61]
25   [71.0, 13.0]    [80, 36]
26   [21.0, 13.0]    [31, 66]
27    [7.0, 40.0]    [37, 86]
28    [46.0, 3.0]    [59, 69]
29   [27.0, 22.0]    [77, 86]
30   [45.0, 41.0]   [83, 106]
31   [10.0, 44.0]    